# The Graph and Database Framework

## Basic Usage

The basic building block of the framework is
the **ProgramNode** which is an abstract class that describes a generic node in the graph.
Two basic types of nodes are **PyNode** and **QuaNode**, those contain Python and Qua programs, respectively.
To start a new node we need to give it a label, a program to be executed, define input/output variables. For example: 

In [1]:
#First some imports
from qualibs.graph import *
from qm.QuantumMachinesManager import QuantumMachinesManager
from qm.qua import *
from qm import LoopbackInterface
from qm import SimulationConfig

Let's say we want our node to take a number and output it's square

In [2]:
a = PyNode('square')
a.program = lambda x: {'y':x**2}
a.input_vars = {'x': 3}
a.output_vars = {'y'}
await a.run_async()


RUNNING PyNode <square>...
DONE running node <square>


Here we used the **await** keyword, which is native in python 3.8 and is used to run code asynchronously, that's how the graph framwork works under the hood. Since this tutorial is written in a Jupyter Notebook one must use the asynchronous version explicitly. The simpler version on PyCharm, for example, would be **a.run()**.
Now, let's explain what we did: 
1. We defined the name/label of the node by a string 'square'.
2. The program must define a dictionary as the output, giving names to all the output variables.
3. The input_vars are defined in a keyword arguments format to be given to the program.
4. The output_vars are a set which contains the variables we would like to store in the result


In [3]:
a.result

{'y': 9}

Another possibility of defining a node in one line:

In [4]:
def add(x, y):
    return {"x":x ,"y": y, "sum": x+y}

sum_node = PyNode("sum_two",add,{'x':2,'y':-3},{'sum','x'})

await sum_node.run_async()
sum_node.result


RUNNING PyNode <sum_two>...
DONE running node <sum_two>


{'sum': -1, 'x': 2}

A nice property of the input_vars attribute is that it can be set using direct assignment by the name of the variable, for example:

In [5]:
sum_node.input_vars.x = 5
sum_node.input_vars.y = 6
await sum_node.run_async()
sum_node.result


RUNNING PyNode <sum_two>...
DONE running node <sum_two>


{'sum': 11, 'x': 5}

Notice: that the **input vars** must match the variables in the **signature** of the program, and the **output vars** the **output dictionary** names.

Now let's create a QuaNode! A QuaNode shares the same attributes as the PyNode but has some additional properties:
1. quantum_machine : defines the QuantumMachine to be used to run the Qua program
2. simulation/execution arguments
3. Whether to simulate_or_execute toggle
4. The program of the QuaNode has somewhat different structure

In [6]:
#import a template config
from qualibs.templates.vanilla_config import config

# open a QM
QMm = QuantumMachinesManager()
QM = QMm.open_qm(config)
sim_args = {'simulate': SimulationConfig(int(1e3))}

2020-11-02 12:57:41,530 - qm - INFO - Performing health check
2020-11-02 12:57:41,533 - qm - INFO - Health check passed
2020-11-02 12:57:41,535 - qm - WARNING - Client's version (0.5.124) does not match the server version (0.5.145).


Now, to define a program for the QuaNode, one must wrap a Qua program in a python function, and the python must return the desired qua program that needs to be executed:

In [7]:
def qua_prog(x):
    
    with program() as prog:
        res = declare(int)
        res_str = declare_stream()
        play('playOp', 'qe1')
        assign(res, x)
        save(res, res_str)
        with stream_processing():
            res_str.save_all('res')
    
    return prog

b = QuaNode('save_x',qua_prog,{'x':5},{'res'})

b.simulation_kwargs = sim_args
b.quantum_machine = QM

await b.run_async()
b.result


SIMULATING QuaNode <save_x>...
2020-11-02 12:57:41,968 - qm - INFO - Flags: 
2020-11-02 12:57:41,969 - qm - INFO - Executing high level program
DONE running node <save_x>


{'res': array([5], dtype=int64)}

Here, the **input_vars** as before must match the **input of the given program**,
and the **output_vars** must match the  saved **stream_processing variables** by the Qua program.

## The Graph

Another basic enetity in the framework is the **ProgramGraph**. This is what links together the nodes decribed above, and what controls the execution and dataflow between the nodes. Let's define a new graph:


In [8]:
g = ProgramGraph('new_graph')

Now suppose we want our graph to have nodes 'a','sum_node' connected such that the output variable y of 'a' will be used as input vaiable x of 'sum_node'.

In [9]:
sum_node.input_vars.y = a.output('y')

Now simply add the nodes to the graph:

In [10]:
g.add_nodes(a,sum_node)

Now to run the graph the same story about async applies here. The graph will be run in an order defined by the dependencies, where independent nodes could run asynchronously.

In [11]:
await g.run_async()


RUNNING PyNode <square>...
DONE running node <square>

RUNNING PyNode <sum_two>...
DONE running node <sum_two>


The result is stored in each of the nodes.

In [12]:
sum_node.result

{'sum': 14, 'x': 5}

Let's add node 'b' to the graph, and use the output of 'a' as the input to node 'b'

In [13]:
b.input_vars.x = a.output("y")

g.add_nodes(b)

await g.run_async()


RUNNING PyNode <square>...
DONE running node <square>

RUNNING PyNode <sum_two>...
DONE running node <sum_two>

SIMULATING QuaNode <save_x>...
2020-11-02 12:57:45,096 - qm - INFO - Flags: 
2020-11-02 12:57:45,097 - qm - INFO - Executing high level program
DONE running node <save_x>


In [14]:
b.result

{'res': array([9], dtype=int64)}

We can see the edges of the graph are constructed automatically from the output flow, but one can add edges manually to ensure a time ordered execution of the nodes. For example, since there's no data flow between 'b' and 'sum_node' one can add an edge from 'b' to 'sum_node' to ensure that 'b' runs before 'sum_node'.

In [15]:
g.add_edges((b,sum_node))
await g.run_async()


RUNNING PyNode <square>...
DONE running node <square>

SIMULATING QuaNode <save_x>...
2020-11-02 12:57:45,817 - qm - INFO - Flags: 
2020-11-02 12:57:45,818 - qm - INFO - Executing high level program
DONE running node <save_x>

RUNNING PyNode <sum_two>...
DONE running node <sum_two>


Now, to vizualize the graph we can use the graph DOT format, and put the string into http://www.webgraphviz.com/ 

In [16]:
print(g.export_dot_graph())

digraph new_graph {{square [shape=box];sum_two [shape=box];save_x [shape=ellipse];};"square" -> "sum_two" [label="['y']"];"square" -> "save_x" [label="['y']"];"sum_two";"save_x" -> "sum_two" [label="!none"];}


One can also pass a **QuaJobNode** from the QuaNode to another node by using **node.qua_job()** and interact with it's QuaJob and QuantumMachine objects directly.

In [17]:
c = PyNode('job_data')
def proccess_job(j):
    # wait for all results
    j.wait()
    # stop job
    j.stop()
    # set/get IO1/2 vars
    j.IO1 = 3
    return {"b_res" : j.get_values('res')}

c.input_vars.j = b.qua_job()
c.program = proccess_job
c.output_vars = {"b_res"}
g.add_nodes(c)
await g.run_async()
print(c.result)


RUNNING PyNode <square>...
DONE running node <square>

SIMULATING QuaNode <save_x>...
2020-11-02 12:57:46,757 - qm - INFO - Flags: 
2020-11-02 12:57:46,758 - qm - INFO - Executing high level program
DONE running node <save_x>

RUNNING PyNode <sum_two>...
DONE running node <sum_two>

RUNNING PyNode <job_data>...
2020-11-02 12:57:46,824 - qm.job - ERROR - Currently not implemented. Will be implemented in future releases.
DONE running node <job_data>
{'b_res': array([9], dtype=int64)}


#### Deepcopy
One can deep copy the nodes and the graph entirely in a working condition by simply using the deepcopy method.

In [18]:
new_a = a.deepcopy()
print(new_a)
print(a)
#The only difference here is the id

{'_id': 2415437485056, '_label': 'square', '_program': <function <lambda> at 0x0000023260AF55E0>, '_input_vars': {'x': 3}, '_output_vars': {'y'}, '_to_run': True, '_metadata_funcs': [], '_result': {'y': 9}, '_start_time': datetime.datetime(2020, 11, 2, 12, 57, 46, 750023), '_end_time': datetime.datetime(2020, 11, 2, 12, 57, 46, 751020), '_type': 'Py', '_save_result_to_db': True, '_job_results': {'y': 9}}
{'_id': 2415435056224, '_label': 'square', '_program': <function <lambda> at 0x0000023260AF55E0>, '_input_vars': {'x': 3}, '_output_vars': {'y'}, '_to_run': True, '_metadata_funcs': [], '_result': {'y': 9}, '_start_time': datetime.datetime(2020, 11, 2, 12, 57, 46, 750023), '_end_time': datetime.datetime(2020, 11, 2, 12, 57, 46, 751020), '_type': 'Py', '_save_result_to_db': True, '_job_results': {'y': 9}}


In [19]:
new_a.input_vars.x = 8
await new_a.run_async()
new_a.result


RUNNING PyNode <square>...
DONE running node <square>


{'y': 64}

In [20]:
new_g = g.deepcopy()
await new_g.run_async()
# access the nodes by label
new_g.nodes_by_label['square'].pop().result

ATTENTION the QuaJob of node <save_x> was NOT copied

RUNNING PyNode <square>...
DONE running node <square>

SIMULATING QuaNode <save_x>...
2020-11-02 12:57:47,682 - qm - INFO - Flags: 
2020-11-02 12:57:47,683 - qm - INFO - Executing high level program
DONE running node <save_x>

RUNNING PyNode <sum_two>...
DONE running node <sum_two>

RUNNING PyNode <job_data>...
2020-11-02 12:57:47,760 - qm.job - ERROR - Currently not implemented. Will be implemented in future releases.
DONE running node <job_data>


{'y': 9}

As we can see, for the QuaNode the job of the execution of the quanutm machine was not copied.
However, the QuantumMachine was copied which means the new QuaNode will run on the same QuantumMachine.

## The Database

Every execution of the graph can be saved in a SQLite database. First, one needs to create a database object by simply specifying the desired location.

In [21]:
g_db = GraphDB("new.db")

Then one can assign the DB object to the graph object, or one can do it per run of the graph.

In [22]:
g.graph_db = g_db;
await g.run_async(verbose=True);

Saving graph <new_graph> to DB at 'new.db'
Saving metadata before running node <square>

RUNNING PyNode <square>...
DONE running node <square>
Saving metadata before running node <save_x>

SIMULATING QuaNode <save_x>...
2020-11-02 12:57:49,060 - qm - INFO - Flags: 
2020-11-02 12:57:49,061 - qm - INFO - Executing high level program
DONE running node <save_x>
Saving metadata before running node <sum_two>

RUNNING PyNode <sum_two>...
DONE running node <sum_two>
Saving metadata before running node <job_data>

RUNNING PyNode <job_data>...
2020-11-02 12:57:49,142 - qm.job - ERROR - Currently not implemented. Will be implemented in future releases.
DONE running node <job_data>
Saving graph <new_graph> results to DB at 'new.db'


Notice the location of the DB

In [23]:
await g.run_async(GraphDB('here.db'),verbose=True);

Saving graph <new_graph> to DB at 'here.db'
Saving metadata before running node <square>

RUNNING PyNode <square>...
DONE running node <square>
Saving metadata before running node <save_x>

SIMULATING QuaNode <save_x>...
2020-11-02 12:57:49,518 - qm - INFO - Flags: 
2020-11-02 12:57:49,518 - qm - INFO - Executing high level program
DONE running node <save_x>
Saving metadata before running node <sum_two>

RUNNING PyNode <sum_two>...
DONE running node <sum_two>
Saving metadata before running node <job_data>

RUNNING PyNode <job_data>...
2020-11-02 12:57:49,591 - qm.job - ERROR - Currently not implemented. Will be implemented in future releases.
DONE running node <job_data>
Saving graph <new_graph> results to DB at 'here.db'


The GraphDB object saves the strucure of the graph, the results of the execution, the scripts of each node, and when possible the script of the graph run. One can decide whether to save a node's result to the DB but toggling **node.save_result_to_db = True/False**

Finally, one can save related metadata about the physical enviroment that is controlled by python, i.e, a temprature sensor, spectrum analyzer, etc.
To do that one needs to define functions for these sources of data in the following manner:

In [25]:
envmodule = {}

# Global enviroment varibles applied to the whole graph
@env_dependency(envmodule)
def volume():
    print("opened volume device")
    return 5
@env_dependency(envmodule)
def pressure():
    print("opened pressure device")
    return 10

def global_metadata(volume,pressure):
    print("Called global metedata")
    v1 = volume()
    v2 = pressure()
    return {"temp": v1*v2}

# Local enviroment variables per node
def local_meta(pressure):
    print("Called local metadata")
    return {'p': pressure()}

new_g_db =  GraphDB("with_metadata.db",global_metadata_funcs=global_metadata,envmodule=envmodule)
c.metadata_funcs = local_meta
await g.run_async(new_g_db,verbose=True);

Saving graph <new_graph> to DB at 'with_metadata.db'
Saving metadata before running node <square>

RUNNING PyNode <square>...
DONE running node <square>
Saving metadata before running node <save_x>

SIMULATING QuaNode <save_x>...
2020-11-02 13:56:13,313 - qm - INFO - Flags: 
2020-11-02 13:56:13,313 - qm - INFO - Executing high level program
DONE running node <save_x>
Saving metadata before running node <sum_two>

RUNNING PyNode <sum_two>...
DONE running node <sum_two>
Saving metadata before running node <job_data>
Called local metadata
opened pressure device

RUNNING PyNode <job_data>...
2020-11-02 13:56:13,548 - qm.job - ERROR - Currently not implemented. Will be implemented in future releases.
DONE running node <job_data>
Saving graph <new_graph> results to DB at 'with_metadata.db'
